In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
import os

from experiment import *

In [7]:
folderpath = '../../Dataset_with_features/'
filename = 'MIT-BIH__DS1_5classes__Fourier.csv'
dataset = pd.read_csv(os.path.join(folderpath, filename));

# Load data ** Chose your path before get stated **

In [2]:
folderpath = '/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Database_extracted/Dataset_with_features/'
filename = 'MIT-BIH__DS1_5classes__Fourier.csv'
dataset = pd.read_csv(os.path.join(folderpath, filename));

In [8]:
dataset.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,target
0,0.140163,0.302116,0.256512,0.199795,0.167456,0.136428,0.092218,0.053257,0.035375,0.022090,...,0.001537,0.003202,0.000649,0.002842,0.002034,0.003303,0.002120,0.001724,0.001094,0
1,0.164694,0.294092,0.248227,0.189410,0.155726,0.119119,0.084469,0.051921,0.026894,0.022658,...,0.001776,0.001309,0.001864,0.002886,0.001600,0.001463,0.001042,0.000314,0.001563,0
2,0.264069,0.336712,0.279655,0.212017,0.169007,0.126995,0.081884,0.039922,0.025413,0.012168,...,0.002457,0.002108,0.001981,0.001673,0.002745,0.001638,0.001236,0.002879,0.002812,0
3,0.072350,0.351005,0.288275,0.218995,0.177219,0.124068,0.074372,0.044149,0.024169,0.016180,...,0.001813,0.001525,0.002232,0.003359,0.001211,0.002223,0.001470,0.002496,0.000781,0
4,0.029850,0.354420,0.295535,0.229964,0.184878,0.129478,0.076174,0.041396,0.027271,0.015194,...,0.002030,0.000666,0.001806,0.002329,0.001273,0.002032,0.001337,0.002298,0.001719,0


# Data preprocessing

In this step we are going standardize our dataset.

In [4]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

Separete features from labels.

In [5]:
data_std = std_scaler.fit_transform(dataset.values[:,:-1])
data_label = dataset.values[:,-1]

# Classifiers specifications

Could be different classifiers, create each one using a different objects

In [6]:
# linear models:
from sklearn.linear_model import SGDClassifier

# Non-parametric models:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Probability distribution Models (might be linear or quadratic):
from sklearn.naive_bayes import GaussianNB

# Non-linear models:
from sklearn.neural_network import MLPClassifier

# Kernel-based models
from sklearn.svm import SVC

##### Model pre-loadings:

# MLP:
mlp_clf_1H = MLPClassifier(solver='adam', alpha=1e-5, learning_rate='adaptive', learning_rate_init = 0.1,
                        max_iter=1300, momentum=0.3, activation = 'relu', power_t=0.5,
                        hidden_layer_sizes=(100,), verbose=0)

mlp_clf_2H = MLPClassifier(solver='adam', alpha=1e-5, learning_rate='adaptive', learning_rate_init = 0.1,
                        max_iter=1300, momentum=0.3, activation = 'relu', power_t=0.5,
                        hidden_layer_sizes=(100, 50), verbose=0)

# SGD:
sgd_clf = SGDClassifier(eta0=0.001, max_iter=1000)

# KNN:
knn_clf = KNeighborsClassifier(n_neighbors=10)

# Random Forest:
rForest_clf = RandomForestClassifier()

# Gaussian Naive-bayes:
naiveBayes_clf = GaussianNB()

# SVM
svm_clf_rbf = SVC(kernel='rbf')
svm_clf_poly = SVC(kernel='poly')
svm_clf_linear = SVC(kernel='linear')

# Classifiers for training an testing

Define our models to train. Import each classifiers into a python dictionary.

In [7]:
classifiers = {'MLP-adam-100H': mlp_clf_1H,
               'MLP-adam-100H50H': mlp_clf_2H,
               'SGD': sgd_clf,
               'KNN_10': knn_clf,
               'Random-Forest': rForest_clf,
               'Naive-Bayes': naiveBayes_clf,
               'SVM-RBF': svm_clf_rbf,
               'SVM-Poly': svm_clf_poly,
               'SVM-Linear': svm_clf_linear,
              }

#classifiers = {'MLP-adam-100H': mlp_clf_1H, 'MLP-adam-100H50H': mlp_clf_2H}

Stratified cross-validation for model selection are going to be used.

In [8]:
kfolds = 8
clf_outputs = run_classifiers(data_std, data_label, classifiers, kfolds)

# Performance assessment

Saving variables in a dictionary:

In [9]:
results = {}
results['train'] = results_clf(5, clf_outputs['train']['true'], clf_outputs['train']['pred'])
results['test'] = results_clf(5, clf_outputs['test']['true'], clf_outputs['test']['pred'])

/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Savel results, models and pipeline to a .pkl file

In [10]:
save_models(classifiers)
save_pipeline(std_scaler)

from sklearn.externals import joblib
joblib.dump(clf_outputs,'teste.pkl')

['teste.pkl']

# Save results to CSVs and figures

In [11]:
export_results(results['test'], 'test')
export_results(results['train'], 'train')

/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/Fourier/Experiment_01_scikitLearn__MLP/experiment.py:236: RuntimeWarning: invalid value encountered in true_divide
  return np.around((confMat / np.sum(confMat,axis=1)[:,None])*100,2)
